# RSA+OAEP vs ElGamal vs ECC-ElGamal Benchmarks

- Device: Dell XPS 15 9570 2018
- OS: UBUNTU 20.04 (stable)
- Memory: 16GiB System Memory, 2x 8GiB SODIMM DDR4 Synchronous 2667 MHz
- Cache: L1 cache -> 384KiB, L2 cache -> 1536KiB, L3 cache -> 9Mib
- Processor: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, 6 Cores, 12 threads
- Graphic Card: Integrated -> UHD Graphics 630 Mobile, Discrete -> GeForce GTX 1050 Ti Mobile

## Imports

In [ ]:
import sys
import time
import warnings
from pathlib import Path

path = Path.cwd().resolve()
warnings.filterwarnings("ignore")

In [ ]:
import plotly.graph_objects as px
from pyinstrument import Profiler as pyProfiler

pyProfiler = pyProfiler()

import psutil
from pyJoules.device import DeviceFactory
from pyJoules.device.rapl_device import RaplCoreDomain
from pyJoules.energy_meter import EnergyMeter

device = DeviceFactory.create_devices([RaplCoreDomain(0)])
meter = EnergyMeter(device)

%load_ext memory_profiler

In [ ]:
from rsa import rsa

sys.path.append("/home/apollo/Desktop/DJSCE/Projects/css_research/ecc_elgamal")
from ecc_elgamal import elgamal as ecc_elg
from ecc_elgamal.cipher.curve import (E222, E382, M383, P256, Curve25519,
                                      secp256k1)

sys.path.append("/home/apollo/Desktop/DJSCE/Projects/css_research/norm_elgamal")
from norm_elgamal import elgamal as norm_elg

In [ ]:
wall_time, mem_use, cpu_use, power_consumed, similarity = [], [], [], [], []

key_sizes = [1024, 2048, 3072, 4096]
message_sizes = ["1", "8", "64", "256", "512"]
messages = [
    b"At.",
    b"Vivamus.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit vivamus.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Praesent eget dapibus elit, in suscipit felis. Etiam enim eros, euismod in sem vitae, porta mattis enim. Integer sed augue eu quam pulvinar semper id nec lorem. Nunc tristique nisi nec sem cras amet.",
    b"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed eu nibh accumsan, pretium lorem sed, fermentum eros. Proin mi massa, hendrerit et bibendum vel, pulvinar eu turpis. Donec vel gravida ipsum, in vehicula odio. Integer non mi tincidunt, pulvinar urna id, gravida ante. Nullam accumsan ac odio eget bibendum. Cras rhoncus turpis et orci imperdiet, facilisis facilisis purus dignissim. Cras pretium sapien sit amet velit condimentum eleifend. Vestibulum consequat metus neque, nec pretium ex tincidunt at.",
]

<br>

## Testing Helper Functions

In [ ]:
def doNothing(tp_time):
    time.sleep(tp_time)

In [ ]:
# Similarity Matching using Damerau Levenshtein Distance
def textSimilarity(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)

    for i in range(-1, lenstr1 + 1):
        d[(i, -1)] = i + 1

    for j in range(-1, lenstr2 + 1):
        d[(-1, j)] = j + 1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1

            d[(i, j)] = min(
                d[(i - 1, j)] + 1,  # deletion
                d[(i, j - 1)] + 1,  # insertion
                d[(i - 1, j - 1)] + cost,  # substitution
            )

            if i and j and s1[i] == s2[j - 1] and s1[i - 1] == s2[j]:
                d[(i, j)] = min(d[(i, j)], d[i - 2, j - 2] + cost)  # transposition

    return round(1 - ((d[lenstr1-1,lenstr2-1])/(lenstr1 if lenstr1 > lenstr2 else lenstr2)), 4)

In [ ]:
def drawGraph(graph_vals: list, key_size: int, save_path: Path, test_param: str, y_val="") -> None:
    """
    Plots graph for function.

    Parameter
    ----------
    graph_vals : list
        Measurement values
    key_size : int
        Size of key
    save_path : pathlib.Path
        Location to save graph image
    test_param : str
        Name of parameter tested
    """
    fig = px.Figure([px.Bar(x=message_sizes, y=graph_vals, text=graph_vals, textposition='auto')])
    fig.update_layout(
    title=f"{test_param}: {key_size} bit key",
    xaxis_title="Key Size(in bits)",
    yaxis_title=y_val,
    marker_color=['deepskyblue', 'cornflowerblue', 'royalblue', 'mediumblue', 'navy'],
    )

    fig.show()
    
    fig.write_image(f"{save_path}/{test_param}_{key_size}.png")

In [ ]:
def benchCode(benchFunc, *args):
    """
    Generalized code for benchmarking cryptographic functions.

    Parameter
    ----------
    benchFunc : function
        The function to be benchmarked.
    """
    curr_time = %timeit -r 5 -o -q -c -p 5 benchFunc(*args)
    
    doNothing(5)
    curr_mem = psutil.Process().memory_info().rss / (1024 * 1024)
    
    doNothing(5)
    curr_cpu_use = psutil.getloadavg()[0] / psutil.cpu_count() * 100
    
    meter.start(tag='foo')
    doNothing(5)
    meter.record(tag='bar')
    benchFunc(*args)
    meter.stop()
    curr_power = meter.get_trace()
    
    doNothing(5)
    plain_text, cipher_text = benchFunc(*args)
    plain_cipher_sim = textSimilarity(str(plain_text), str(cipher_text))
    
    wall_time.append(round(curr_time.average, 3))
    mem_use.append(round(curr_mem, 3))
    cpu_use.append(round(curr_cpu_use, 3)) 
    power_consumed.append(curr_power) Please check which value to append
    similarity.append(round((plain_cipher_sim*10), 3))

<br>

## RSA with OAEP

In [ ]:
name = "rsa/results"

try:
    dir_path = path.joinpath(name)
    dir_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    pass

In [ ]:
for key in key_sizes:
    del wall_time[:], mem_use[:], cpu_use[:], power_consumed[:], similarity[:] 
    for plain_text in messages:
        try:
            benchCode(rsa.caller, plain_text, key)
        except (ValueError, OverflowError) as e:  # if it cannot encrypt put value as 0
            print(e, key)
            wall_time.append(0)
            mem_use.append(0)
            cpu_use.append(0) 
            power_consumed.append(0)
            similarity.append(0)
    
    drawGraph(wall_time, key, dir_path, "Execution Time", "seconds")
    drawGraph(mem_use, key, dir_path, "Memory Used", "MB")
    drawGraph(cpu_use, key, dir_path, "CPU Used", "% Used")
    drawGraph(power_consumed, key, dir_path, "Power Consumed")
    drawGraph(similarity, key, dir_path, "Plain and Cipher Text Similarity", "% Similar")

<br>

## Plain ElGamal

In [ ]:
name = "norm_elgamal/results"

try:
    dir_path = path.joinpath(name)
    dir_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    pass

In [ ]:
for key in key_sizes:
    del wall_time[:], mem_use[:], cpu_use[:], power_consumed[:], similarity[:] 
    for plain_text in messages:
        try:
            benchCode(norm_elg.caller, plain_text.decode("utf-8"), key)
        except (ValueError, OverflowError) as e:  # if it cannot encrypt put value as 0
            print(e, key)
            wall_time.append(0)
            mem_use.append(0)
            cpu_use.append(0) 
            power_consumed.append(0)
            similarity.append(0)
    
    drawGraph(wall_time, key, dir_path, "Execution Time", "seconds")
    drawGraph(mem_use, key, dir_path, "Memory Used", "MB")
    drawGraph(cpu_use, key, dir_path, "CPU Used", "% Used")
    drawGraph(power_consumed, key, dir_path, "Power Consumed")
    drawGraph(similarity, key, dir_path, "Plain and Cipher Text Similarity", "% Similar")

<br>

## ECC ElGamal

In [ ]:
name = "ecc_elgamal/results"

try:
    dir_path = path.joinpath(name)
    dir_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    pass

In [ ]:
for key in (E222, E382, M383, P256, Curve25519, secp256k1):
    del wall_time[:], mem_use[:], cpu_use[:], power_consumed[:], similarity[:] 
    for plain_text in messages:
        try:
            benchCode(ecc_elg.caller, plain_text, key)
        except (ValueError, OverflowError) as e:  # if it cannot encrypt put value as 0
            print(e, key)
            wall_time.append(0)
            mem_use.append(0)
            cpu_use.append(0) 
            power_consumed.append(0)
            similarity.append(0)

    drawGraph(wall_time, str(key), dir_path, "Execution Time", "seconds")
    drawGraph(mem_use, str(key), dir_path, "Memory Used", "MB")
    drawGraph(cpu_use, str(key), dir_path, "CPU Used", "% Used")
    drawGraph(power_consumed, str(key), dir_path, "Power Consumed")
    drawGraph(similarity, str(key), dir_path, "Plain and Cipher Text Similarity", "% Similar")